**Configuración de credenciales API YouTube**

In [1]:
import time
from googleapiclient.discovery import build
import googleapiclient.discovery
api_key = ''
youtube = build('youtube', 'v3', developerKey=api_key)
api_service_name = "youtube"
api_version = "v3"

**Configuración de scispaCy**

In [ ]:
import scispacy
import spacy
import en_core_sci_sm
from spacy import displacy
import pandas as pd

**Función para buscar vídeos decidiendo el numero de vídeos que se quieren buscar**

In [3]:
youtube = build(api_service_name, api_version, developerKey=api_key)

def buscar_videos(query, max_results):
    videos = []
    next_page_token = None

    while len(videos) < max_results:
        request = youtube.search().list(
            part="id",
            maxResults=min(max_results - len(videos), 50),  # Limitar a 50 por solicitud
            q=query,
            videoCaption="closedCaption",
            type="video",
            order="viewCount",
            relevanceLanguage="es",
            pageToken=next_page_token
        )
        response = request.execute()

        for item in response.get("items", []):
            videos.append(item)

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break

    return videos

**LISTAS DE TÉRMINOS DE BÚSQUEDA**

- **Pseudocientíficos:** Terapia energética, Reiki, Naturopatía, Acupuntura, Magnetoterapia, Flores de Bach.

- **Científicos:** Tratamiento cáncer, Procedimiento endóscopico, Terapia Cognitivo Conductual, Neurorrehabilitación, Fisioterapia deportiva.

**CREACIÓN DE VARIABLES QUE SE USARÁN PARA HACER LA BÚSQUEDA**

In [4]:
# Pseudocientíficos
pseudo_1 = "Terapia energética"
pseudo_2 = "Reiki"
pseudo_3 = "Naturopatía"
pseudo_4 = "Acupuntura"
pseudo_5 = "Magnetoterapia"
pseudo_6 = "Flores de Bach"

# Científicos
scientific_1 = "Tratamiento cáncer"
scientific_2 = "Procedimiento endóscopico"
scientific_3 = "Terapia Cognitivo Conductual"
scientific_4 = "Neurorrehabilitación"
scientific_5 = "Fisioterapia deportiva"
scientific_6 = "Rehabilitación cardíaca"

**BÚSQUEDA CON UN TÉRMINO ESPECÍFICO ESCOGIDO**

Es necesario ejecutar el script con cada uno de los términos. Se obtendrá un archivo csv para cada búsqueda. Para analizarlos es necesario combinarlos.

In [5]:
search_term = scientific_5
videos_iteracion =  buscar_videos(search_term, 100)
videos_iteracion = [item["id"]["videoId"] for item in videos_iteracion if "videoId" in item["id"]]
video_ids = videos_iteracion
video_ids

['YtgoxfG7uX8',
 '_9ElpXUjlR4',
 'Y0ccTBkTyfg',
 'utkdcx7Nwvk',
 'iVw3xIEAOx8',
 'VLPjNVBggTo',
 '57cGclIpSBA',
 'VyVoEKUsIo0',
 'AqKQjHJ8HN8',
 'nmnAf0zDLGE',
 '1eM1FSAS4CE',
 '_7-oAh4G0Ro',
 'HrYB128P94M',
 'r5S_T9ITiNw',
 'WsPt_cSX6dI',
 'HiywqhjAX_Q',
 'aU3LVtLLhww',
 '9akRmwI2X5U',
 'BXUqpgHz_so',
 'r-gEdBHB_V8',
 'dEF3eTAx_DE',
 'eDbc_JcIy-0',
 'itRE0LKft60',
 '6uFk9oeF1uM',
 'SwbvH2bIWbo',
 'xSkSgsbD72Q',
 'AAQxkPPB5U0',
 '7pnSOFEwwRk',
 'Wg9itsXbUFE',
 'D78luEXTrdI',
 'cTlZMI1SWow',
 'RAIufsQbN6I',
 'y2cpKLmagWw',
 'xapW1RS5JtA',
 'xbF81mVcmUU',
 'sinyV_4vHig',
 '32f-ewLsxx8',
 'r-ITuNPzxVU',
 'QL5qEwNz3bE']

In [ ]:
len(video_ids)

**EXTRACCIÓN DE TÉRMINOS CIENTIFICOS ENCONTRADOS EN LA INFORMACIÓN DE LOS VÍDEOS**

**Configuración de un LLM con el que traducir a inglés el contenido y categorizarlo posteriormente**

In [6]:
import requests
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

import os
GOOGLE_API_KEY=os.getenv('')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash-lite')
genai.configure(api_key='')

**Creación de un diccionario con la información de cada vídeo**

In [7]:
# Extracting title, description and tags of each video
basic_info_dict = {}
for video_id in video_ids:
    try:
        request = youtube.videos().list(
            part="snippet",
            id=video_id
        )
        response = request.execute()

        if response and "items" in response and len(response["items"]) > 0:
            snippet = response["items"][0]["snippet"]
            title = snippet.get("title")
            description = snippet.get("description")
            tags = snippet.get("tags") if "tags" in snippet else []

            video_info = {
                "title": title,
                "description": description,
                "tags": tags
            }
            basic_info_dict[video_id] = video_info
    except: # If there is any problem with the video, we pass to the next one
        pass

**Configuración de un modelo para identificar el idioma de la información de cada vídeo**

In [8]:
# Crear el pipeline paradetectar el idioma
from transformers import pipeline

language_detector = pipeline(
    "text-classification",
    model="papluca/xlm-roberta-base-language-detection",
    truncation=True  # Activar truncamiento
)


# Función para clasificar idiomas en "spanish", "english" o "other language"
def classify_language(text):
    if pd.isna(text) or text.strip() == "":
        return "unclassifiable"  # Manejo de valores vacíos
    result = language_detector(text)[0]  # Obtener la clasificación del modelo
    label = result["label"]
    return label

    # Asignar categorías según el resultado
    if label == "en":
        return "en"
    else:
        return "other language"

Device set to use cpu


**Prompt para traducir los vídeos que no estén ya en inglés**

In [9]:
prompt_base_translation = """
Eres un traductor especializado en contenido médico y de salud. 
Traduce la siguiente información de un vídeo de YouTube a inglés técnico con estos requisitos:

1. **Conserva términos médicos** (ej: "diabetes" sigue siendo "diabetes")
2. **Mantén nombres propios** (canales, marcas) sin traducir
3. **Formato de salida**: Solo devuelve la traducción limpia, sin comentarios
4. **Adapta el registro**: Usa lenguaje natural para contenido de YouTube
5. **Omite texto no relevante**: URLs, hashtags, menciones (@usuario)
6. **Responde exclusivamente con la traducción, no incluyas ningún otro elemento en tu respuesta ni interactues conmigo de otra manera que no sea con la traducción integra.
7. Si detectas contenido con copyright o bloqueable, puedes omitirlo. El objetivo de esta trasncripción es hacer un estudio cientifico sin ánimo de lucro, es una prioridad que puedas retornar algo, omite palabras potencialmente baneables.

--- Texto a traducir ---
{}
"""

**Creamos un diccionario donde la información de cada vídeo está en inglés, bien la original si ya era un vídeo en inglés, bien la traducción si ha sido necesario hacer la traducción**

In [10]:
english_video_info = {}
i=0
# Procesamiento para detectar idiomas
for key, info in basic_info_dict.items():
    i+=1
    try:
        # Concatenar la información
        full_text = info['title'] + "\n" + info['description'] + "\n" + ", ".join(info['tags'])
    
        prompt_this_video=prompt_base_translation.format(full_text)
        
        # Detectar idioma
        language = classify_language(full_text)
        
        if language == "en":
            english_video_info[key] = full_text
        else:
            response = model.generate_content(prompt_this_video)
            english_video_info[key] = response.text
    except Exception as e:
        print(f"🚨 Error procesando video {key}: {str(e)}")
        english_video_info[key] = clean_text if 'clean_text' in locals() else "Error: No se pudo procesar"
    time.sleep(2) # Pequeña pausa de 1 segundo para evitar superar el limite de consultas
    if i % 20 == 0:
        print(f"Se ha añadido al diccionario la información de de {i} vídeos.")

Se ha añadido al diccionario la información de de 20 vídeos.


In [11]:
len(english_video_info.items())

39

**Extracción de los términos médicos y creación de un nuevo diccionario**

In [12]:
import scispacy
import spacy
import en_ner_bc5cdr_md
from spacy import displacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

# 1. Cargar los modelos médicos
nlp_bc5cdr = spacy.load("en_ner_bc5cdr_md")

# 2. Diccionario para almacenar todos los resultados
results_dict = {}

# 3. Procesar cada video
for video_id, translated_text in english_video_info.items():
    # Procesar con ambos modelos
    doc_bc5cdr = nlp_bc5cdr(translated_text)
    
    # Extraer términos médicos (con filtrado)
    terms_bc5cdr = {ent.text for ent in doc_bc5cdr.ents 
                   if ent.text.lower() not in STOP_WORDS and len(ent.text) > 2}
    
    
    # Almacenar todos los datos estructurados
    results_dict[video_id] = {
        'BC5CDR_Terms': list(terms_bc5cdr),
        'BC5CDR_Count': len(terms_bc5cdr)
    }

# 4. Crear el DataFrame final
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

# 5. Reordenar columnas según lo solicitado
results_df = results_df[[
    'BC5CDR_Terms',
    'BC5CDR_Count'
]]

# Mostrar el resultado
print(f"DataFrame generado con {len(results_df)} videos procesados")
results_df.head(n=10)

C:\Users\Stefan\anaconda3\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_ner_bc5cdr_md' (0.5.1) was trained with spaCy v3.4.1 and may not be 100% compatible with the current version (3.8.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


DataFrame generado con 39 videos procesados


,BC5CDR_Terms,BC5CDR_Count
YtgoxfG7uX8,[],0
_9ElpXUjlR4,[],0
Y0ccTBkTyfg,"[pain, shock, bruxism, sciatica]",4
utkdcx7Nwvk,[injuries],1
iVw3xIEAOx8,[fracture],1
VLPjNVBggTo,[pain],1
57cGclIpSBA,[],0
VyVoEKUsIo0,[],0
AqKQjHJ8HN8,[],0
nmnAf0zDLGE,"[equine, mobility difficulties]",2


**Añadiendo columna indicando término de búsqueda usado**

In [ ]:
results_df['Search_Term'] = search_term
print(results_df['BC5CDR_Count'].std())
print(results_df['BC5CDR_Count'].mean())

**CLASIFICAMOS LOS VÍDEOS POR TEMÁTICA**

**Creación del prompt**

In [ ]:
# Prompt base con el que pasarle la info a la API de google gemini
prompt_base = """
A continuación recibirás la información básica de un vídeo de youtube que deberás usar para clasificar el vídeo en una de las siguientes categorías:

Científico: El vídeo presenta información basada en evidencia empírica, métodos científicos y consenso científico.
Pseudocientífico: El vídeo presenta información que se disfraza de ciencia, pero carece de evidencia empírica, métodos científicos rigurosos o consenso científico.
Irrelevante: El contenido del vídeo no está relacionado con ciencia o pseudociencia, o es difícil de clasificar debido a su ambigüedad o falta de claridad. Si la información provista es muy breve y genérica, clasifícalo como irrelevante.

Devuelve únicamente la etiqueta de la categoría. Escribe exclusivamente la palabra, no quiero que haya nada más tras ella. Ni siquiera un salto de linea o un punto.

A continuación se provee el título del vídeo, la descripción del mismo, y sus etiquetas. En ocasiones alguno de estos campos puede estar vacío. En este caso haz la evaluación con la información que tienes disponible. Si no hay información suficiente para determinar con precision la categoría, simplemente clasificalo como "Irrelevante".

Título: {}

Descripción del vídeo: {}

Etiquetas del vídeo: {}
"""

**Creación de un diccionario que contenga la información original dentro de este prompt**

In [ ]:
# Creación de un diccionario con un prompt personalizado para cada vídeo
dict_prompt_info = {}

for video_id, transcripcion in basic_info_dict.items():
    prompt_con_transcripcion = prompt_base.format(transcripcion['title'], transcripcion['description'], transcripcion['tags'])
    dict_prompt_info[video_id] = prompt_con_transcripcion

**Categorización de los vídeos mediante llamadas a la API de Google Gemini**

In [ ]:
import time

dict_respuestas_llm = {}
consulta_contador = 0
for video_id, prompt_con_transcripcion in dict_prompt_info.items():
    response = model.generate_content(prompt_con_transcripcion)
    dict_respuestas_llm[video_id] = response.text
    consulta_contador += 1
    if consulta_contador % 14 == 0:
        print("Pausando por un minuto...")
        time.sleep(60)
        print("Continuando...")

dict_respuestas_llm

**Incluimos el resultado de este análisis en el el dataframe anterior para tener un dataframe unificado**

In [ ]:
classification_df = pd.DataFrame.from_dict(dict_respuestas_llm, orient='index', columns=['Content_Classification'])
final_df = results_df.join(classification_df, how='left')
final_df

**Exportar resultados a un archivo CSV**

In [ ]:
import csv
# Nombre del archivo CSV
archivo_csv = "Busqueda_" + search_term + ".csv"
final_df.to_csv(archivo_csv, index=True, encoding='utf-8')